Our first presentation was going to answer the question, does healthy equate to happy?  We had great ideas in regards to the data we wanted to collect, as the first go around on the optional data sets and API's appeared to have the data we were looking to measure. Upon further investigatioin,  the step counts, physical stats and gender information was a bit harder to pull from an API than we thought.

Hence, we are now working to see if we can use data provided by the census API, a Kaggle data series that contained over 2,000 Chipotle stores as well as a sample set of data from Kaggle regarding other fast food locations, to see if we can predict what factors may influence a Chipotle location. Originally we had a few different years of information, but because we could not obtain the 2019 census data, we had enough with 2018 we used that year as our basis.

Kaggle data: https://www.kaggle.com/jeffreybraun/chipotle-locations
Sample data from other fast food locations: https://www.kaggle.com/datafiniti/fast-food-restaurants

First we pulled and cleaned all of the data.

In [1]:
import pandas as pd
import numpy as np
import requests
from census import Census

# Census API Key
from config import api_key
c = Census(api_key, year=2018)

## Census Data

In [2]:
# Run Census Search to retrieve data on all zip codes (2017 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E"), {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})

In [3]:
# Final DataFrame
census_pd = census_pd[["Zipcode", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count"]]

# Visualize
print(len(census_pd))
census_pd.head()

33120


,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count
0,00601,17242.0,40.5,13092.0,6999.0,10772.0
1,00602,38442.0,42.3,16358.0,9277.0,19611.0
2,00603,48814.0,41.1,16603.0,11307.0,24337.0
3,00606,6437.0,43.3,12832.0,5943.0,4163.0
4,00610,27073.0,42.1,19309.0,10220.0,11724.0


In [4]:
# Add Poverty Rate at the end
census_pd["Poverty Rate"]=census_pd["Poverty Count"]*100/ census_pd["Population"]


In [5]:
# Drop all NaN data
census_pd.dropna(subset=['Household Income', 'Per Capita Income', 'Poverty Rate'], inplace=True)
print(len(census_pd))

32456


We would need latitude and longitude information for a map later referenced.  A pull from the google places API was not working, and I did not want to run up my API account so I used the referenced link to obtain the information I could then add into the census data. https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/table/

In [6]:
census_pd['Zipcode'] = census_pd.Zipcode.astype(float)
census_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32456 entries, 0 to 33084
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Zipcode            32456 non-null  float64
 1   Population         32456 non-null  float64
 2   Median Age         32456 non-null  float64
 3   Household Income   32456 non-null  float64
 4   Per Capita Income  32456 non-null  float64
 5   Poverty Count      32456 non-null  float64
 6   Poverty Rate       32456 non-null  float64
dtypes: float64(7)
memory usage: 2.0 MB


In [7]:
# Add Latitude and Longitude to census data for later use
latlong ="../Resources/us-zip-code-latitude-and-longitude.csv"
# Read files into Pandas DF
latlong= pd.read_csv(latlong)

In [8]:
print(len(latlong))
latlong.head()
latlong.columns

43191


Index(['Zipcode ', 'City', 'State', 'Latitude', 'Longitude', 'Timezone',
       'Daylight savings time flag', 'geopoint'],
      dtype='object')

In [9]:
# There was an additional space in the Zipcode series from the latitidue file so it had to be renamed prior to merging
latlong=latlong.rename(columns={'Zipcode ': 'Zipcode'})
latlong.head(2)

,Zipcode,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
0,210,Portsmouth,NH,43.005895,-71.013202,-5,1,43.005895
1,211,Portsmouth,NH,43.005895,-71.013202,-5,1,43.005895


In [11]:
# merge with census data
clean_2018_census_pd = pd.merge(census_pd, latlong,on='Zipcode')
clean_2018_census_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32242 entries, 0 to 32241
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Zipcode                     32242 non-null  float64
 1   Population                  32242 non-null  float64
 2   Median Age                  32242 non-null  float64
 3   Household Income            32242 non-null  float64
 4   Per Capita Income           32242 non-null  float64
 5   Poverty Count               32242 non-null  float64
 6   Poverty Rate                32242 non-null  float64
 7   City                        32242 non-null  object 
 8   State                       32242 non-null  object 
 9   Latitude                    32242 non-null  float64
 10  Longitude                   32242 non-null  float64
 11  Timezone                    32242 non-null  int64  
 12  Daylight savings time flag  32242 non-null  int64  
 13  geopoint                    322

In [12]:
# made all Zipcodes integers instead of floats
clean_2018_census_pd['Zipcode']= clean_2018_census_pd['Zipcode'].astype(int)
clean_2018_census_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32242 entries, 0 to 32241
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Zipcode                     32242 non-null  int32  
 1   Population                  32242 non-null  float64
 2   Median Age                  32242 non-null  float64
 3   Household Income            32242 non-null  float64
 4   Per Capita Income           32242 non-null  float64
 5   Poverty Count               32242 non-null  float64
 6   Poverty Rate                32242 non-null  float64
 7   City                        32242 non-null  object 
 8   State                       32242 non-null  object 
 9   Latitude                    32242 non-null  float64
 10  Longitude                   32242 non-null  float64
 11  Timezone                    32242 non-null  int64  
 12  Daylight savings time flag  32242 non-null  int64  
 13  geopoint                    322

In [13]:
# rename columns for easier merging with restaurant data
clean_2018_census_pd = clean_2018_census_pd[['Zipcode','Latitude', 'Longitude','Population', 'Median Age', 'Household Income', 'Per Capita Income', 'Poverty Count', 'Poverty Rate']]
clean_2018_census_pd

,Zipcode,Latitude,Longitude,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate
0,601,18.180103,-66.74947,17242.0,40.5,13092.0,6999.0,10772.0,62.475351
1,602,18.363285,-67.18024,38442.0,42.3,16358.0,9277.0,19611.0,51.014515
2,603,18.448619,-67.13422,48814.0,41.1,16603.0,11307.0,24337.0,49.856599
3,606,18.182151,-66.95880,6437.0,43.3,12832.0,5943.0,4163.0,64.672984
4,610,18.288319,-67.13604,27073.0,42.1,19309.0,10220.0,11724.0,43.305138
...,...,...,...,...,...,...,...,...,...
32237,99922,55.209339,-132.82545,330.0,39.5,34028.0,18213.0,129.0,39.090909
32238,99925,55.555164,-133.07316,927.0,43.6,57375.0,25840.0,172.0,18.554477
32239,99926,55.123897,-131.56883,1635.0,34.5,53409.0,22453.0,235.0,14.373089
32240,99927,56.337957,-133.60689,38.0,55.5,-666666666.0,13658.0,28.0,73.684211


### The next three cells came from my tutor, who had issues with zip codes in a previous project
https://github.com/ealong/Springboard-Capstone1/blob/master/capstone1-wrangling.ipynb

In [15]:
# Converted to string because restaurant data information was listed as string and could not merge properly
# Adjusted series name to allow proper merge as well
# Confirm number of zip codes with less than 5 digits
clean_2018_census_pd['Zipcode'] = clean_2018_census_pd['Zipcode'].astype(str)
clean_2018_census_pd = clean_2018_census_pd.rename(columns={'Zipcode': 'Zip Code'})
census_2018_badzip= clean_2018_census_pd[clean_2018_census_pd['Zip Code'].str.len()<5]
print(census_2018_badzip)

     Zip Code   Latitude  Longitude  Population  Median Age  Household Income  \
0         601  18.180103  -66.74947     17242.0        40.5           13092.0   
1         602  18.363285  -67.18024     38442.0        42.3           16358.0   
2         603  18.448619  -67.13422     48814.0        41.1           16603.0   
3         606  18.182151  -66.95880      6437.0        43.3           12832.0   
4         610  18.288319  -67.13604     27073.0        42.1           19309.0   
...       ...        ...        ...         ...         ...               ...   
2509     8887  40.525361  -74.79632      1004.0        44.7           89674.0   
2510     8889  40.612851  -74.76682      9562.0        50.1          135601.0   
2511     8901  40.488304  -74.44775     57659.0        22.8           42185.0   
2512     8902  40.453131  -74.48287     41135.0        37.0           94490.0   
2513     8904  40.500254  -74.42570     13992.0        35.2           80846.0   

      Per Capita Income  Po

In [16]:
#Research confirms that for the cities and states with zip codesless than 5 digits, the zip codes begin with 0.
#Excel truncates the leading zero. (#e.g. 06001 becomes 6001 in Excel) 

def format_postal_codes(row):
    zipcode= row['Zip Code']
    return zipcode[:5].zfill(5)

In [18]:
# Fix zip codes to have 5 digits
clean_2018_census_pd['Zip Code'] = clean_2018_census_pd.apply(format_postal_codes, axis=1)

census_2018_badzip = clean_2018_census_pd[clean_2018_census_pd['Zip Code'].str.len()<5]
print(census_2018_badzip)

Empty DataFrame
Columns: [Zip Code, Latitude, Longitude, Population, Median Age, Household Income, Per Capita Income, Poverty Count, Poverty Rate]
Index: []


In [19]:
# export to CSV
clean_2018_census_pd.to_csv("../Resources/census_data_clean_2018.csv", encoding="utf-8", index=False)

## Restaurant Data

In [20]:
# load all restaurant first and clean
fast_food_restaurants_2018 = "../Resources/Fast_Food_Restaurants_2018.csv"
# Read files into Pandas DF
Rest_2018 = pd.read_csv(fast_food_restaurants_2018)

In [21]:
# remove columns we will not need
Rest_2018=Rest_2018.drop(['id','sourceURLs','websites'], axis =1)
Rest_2018.columns

Index(['dateAdded', 'dateUpdated', 'address', 'categories', 'city', 'country',
       'keys', 'latitude', 'longitude', 'name', 'postalCode', 'province'],
      dtype='object')

In [22]:
# reorganize/rename columns
Rest_2018 = Rest_2018.rename(columns={'name':'Name', 'address': 'Address', 'city': 'City', 'country': 'Country', 'latitude': 'Latitude', 'longitude':'Longitude', 'postalCode': 'Zip Code',\
                                     'province': 'State'})
Rest_2018 = Rest_2018[['Name', 'Address', 'City','State', 'Zip Code','Country', 'Latitude', 'Longitude']]
Rest_2018.head()

,Name,Address,City,State,Zip Code,Country,Latitude,Longitude
0,SONIC Drive In,800 N Canal Blvd,Thibodaux,LA,70301,US,29.814697,-90.814742
1,SONIC Drive In,800 N Canal Blvd,Thibodaux,LA,70301,US,29.814697,-90.814742
2,Taco Bell,206 Wears Valley Rd,Pigeon Forge,TN,37863,US,35.803788,-83.580553
3,Arby's,3652 Parkway,Pigeon Forge,TN,37863,US,35.782339,-83.551408
4,Steak 'n Shake,2118 Mt Zion Parkway,Morrow,GA,30260,US,33.562738,-84.321143


In [23]:
# get unique names in list and sort alphabetically to assist with cleaning
names = Rest_2018['Name'].unique()
names.sort()
print(len(names))
print(names)

571
['7-Eleven' '90 Miles Cuban Cafe' 'A&W/Long John Silvers'
 "A's Ace Burgers" 'Abruzzi Pizza' 'Acropolis Gyro Palace'
 'Adobe Cantina Salsa' 'Ak Buffet' 'All About Burger'
 'Ameci Pizza & Pasta' "Andy's Frozen Custard" "Angelina's Pizza"
 "Ani's Chicken" "Arby's" "Arby's - Closed" 'Arbys' 'Arctic Circle'
 "Arni's Crawfordsville" "Arthur Treacher's Fish & Chips" 'Au Bon Pain'
 "Auntie Anne's" "Auntie Anne's Pretzels" 'Ay Jalisco' 'B Good'
 'B&D Burgers' "B's Pizza" 'B.GOOD' 'BURGERFI' 'Back Yard Burgers'
 'Backyard Grill' "Bad Daddy's Burger Bar" 'Baja Fresh Mexican Grill'
 "Baker's Drive Thru" "Baker's Drive-thru" 'Band Box Diner' 'Barberitos'
 "Barro's Pizza" 'Baskin-Robbins' 'Batter Up' 'BeanThru' 'Beef Villa'
 "Bell's Pizza" 'Belleria Pizzeria' "Ben & Jerry's" "Ben and Jerry's"
 "Bennidito's Pizza" 'Best Gyros' 'Big House Burgers' 'Big Island Pizza'
 'Bistro To Go' 'Blackies Hotdog Stand' 'Blackjack Pizza'
 "Blake's Lotaburger" 'Blimpie' 'Blue Coast Burrito' 'Boardwalk Grill'
 'B

In [24]:
# Rename all locations that were duplicate but had different capitalizations, punctuations, etc
Rest_2018['Name'] = Rest_2018['Name'].replace({"Arby's - Closed":"Arby's",'Arbys': "Arby's", "Auntie Anne's": "Auntie Anne's Pretzels","Baker's Drive Thru":"Baker's Drive-thru","Ben and Jerry's": "Ben & Jerry's",'Bob Evans Restaurant':'Bob Evans',"Bojangles' Famous Chicken 'n Biscuits": 'Bojangles', 'Burger King¬Æ': 'Burger King','Capri Italian Restaurant':'Capri Restaurant',"Carl's Jr.":"Carl's Jr","Carl's Jr. / Green Burrito":"Carl's Jr","Carl's Jr / Green Burrito":"Carl's Jr",'Caseys Carry Out Pizza':"Casey's General Store","Charley's Grilled Subs":'Charleys Philly Steaks',"Checker's Pizza":'Checkers','Chick-fil-A':'Chick-Fil-A', 'Chipotle':'Chipotle Mexican Grill','Dairy Queen (Treat Only)':'Dairy Queen','Dunkin Donuts':"Dunkin' Donuts",'Five Guys Burgers Fries':'Five Guys', 'Five Guys Burgers And Fries':'Five Guys',"Foster's Freeze":'Fosters Freeze', "Hardee's":'Hardees',"Hardee's / Red Burrito":'Hardees', "Hardee's/red Burrito":'Hardees','Jack in the Box':'Jack In The Box', 'Jack in the Box -':'Jack In The Box'})
Rest_2018['Name'] = Rest_2018['Name'].replace({"Jimmy John's":'Jimmy Johns','KFC - Kentucky Fried Chicken':'KFC','KFC/AW':'KFC','KFC/Long John Silvers':'KFC', 'KFC/Taco Bell':'KFC'})
Rest_2018['Name'] = Rest_2018['Name'].replace({"Little Caesar's Pizza":'Little Caesars', 'Little Caesars Pizza':'Little Caesars',"Long John Silver's / AW" 'Long John Silvers':"Long John Silver's",'Long John Silvers / A&W':"Long John Silver's",'Mc Donalds':"McDonald's","McDonald's of Rolesville":"McDonald's",'McDonalds':"McDonald's","McDonalds's":"McDonald's","Mcdonald's":"McDonald's",'Mcdonalds':"McDonald's",'Mcdonalds Whitehouse':"McDonald's"})
Rest_2018['Name'] = Rest_2018['Name'].replace({'Panda Express Innovation Kitchen':'Panda Express' ,'PepperJax Grill':'Pepperjax Grill',"Popeye's Louisiana Kitchen":'Popeyes','Popeyes Chicken & Biscuits':'Popeyes','Popeyes Chicken Biscuits':'Popeyes','Popeyes Louisiana Kitchen':'Popeyes'})
Rest_2018['Name'] = Rest_2018['Name'].replace({'QDOBA Mexican Eats':'Qdoba Mexican Grill', 'Qdoba Mexican Eats':'Qdoba Mexican Grill', "Quizno's": 'Quiznos', 'Quiznos Sub':'Quiznos',"Raising Cane's":'Raising Canes', "Raising Cane's Chicken Fingers":'Raising Canes'})
Rest_2018['Name'] = Rest_2018['Name'].replace({"Rally's Hamburgers":"Rally's", 'Rallys':"Rally's",'Roma Pizza':'Romas Pizza','SONIC Drive In': 'SONIC Drive-In', 'SUBWAY¬Æ':'SUBWAY'})
Rest_2018['Name'] = Rest_2018['Name'].replace({'Sonic':'SONIC Drive-In', "Sonic America's Drive-In":'SONIC Drive-In','Sonic Drive In':'SONIC Drive-In','Sonic Drive in':'SONIC Drive-In','Sonic Drive-In':'SONIC Drive-In',"Steak 'n Shake":'Steak N Shake','Taco Bell / KFC':'Taco Bell' ,'Taco Bell/KFC':'Taco Bell','Taco Bell/Pizza Hut':'Taco Bell'})
Rest_2018['Name'] = Rest_2018['Name'].replace({ 'Wienerschitzel':'Wienerschnitzel','Wingstop Restaurant':'Wingstop',"Wolf's Dairy Queen" :'Dairy Queen',"Zaxby's Chicken Fingers & Buffalo Wings": "Zaxby's",'b.good':'B.GOOD', 'A&W/Long John Silvers':"Long John Silver's"})
# After going through team decided to rename Pizza Hut, Taco Bell and KFC to YUM Brands
Rest_2018['Name'] = Rest_2018['Name'].replace({'Pizza Hut': 'YUM Brands', 'KFC': 'YUM Brands','Taco Bell': 'YUM Brands'})

In [25]:
print(len(Rest_2018))

10000


In [26]:
# Remove locations that are not food related 'Walmart Supercenter''T-Mobile''Hilton Boston Logan Airport''7-Eleven'
# https://thispointer.com/python-pandas-how-to-drop-rows-in-dataframe-by-conditions-on-column-values/
indexNames= Rest_2018[(Rest_2018['Name']=='Walmart Supercenter')].index
Rest_2018.drop(indexNames, inplace=True)

In [27]:
indexNames= Rest_2018[(Rest_2018['Name']=='T-Mobile')].index
Rest_2018.drop(indexNames, inplace=True)

In [28]:
indexNames= Rest_2018[(Rest_2018['Name']=='Hilton Boston Logan Airport')].index
Rest_2018.drop(indexNames, inplace=True)

In [29]:
indexNames= Rest_2018[(Rest_2018['Name']=='7-Eleven')].index
Rest_2018.drop(indexNames, inplace=True)
print(len(Rest_2018))
Rest_2018['Name'].nunique()

9977


491

In [30]:
# find number of entries for each location
Rest_2018['Name'].value_counts()

McDonald's                      1953
YUM Brands                      1442
Burger King                      834
Subway                           776
Arby's                           669
                                ... 
Price's Chicken Coop               1
Chicken Mary's                     1
Sam's Pizza Restaurant             1
Sweet Magnolia's Cafe Bakery       1
Lobster Hut                        1
Name: Name, Length: 491, dtype: int64

In [31]:
# Remove all locations that have less than 50 entries
# https://www.thetopsites.net/article/58467286.shtml
threshold = 50
clean_rest_2018 = Rest_2018[Rest_2018.groupby('Name')['Name'].transform('count')>threshold].copy()
print(len(clean_rest_2018))
clean_rest_2018['Name'].value_counts()

8526


McDonald's         1953
YUM Brands         1442
Burger King         834
Subway              776
Arby's              669
Wendy's             628
Jack In The Box     332
Chick-Fil-A         228
Dairy Queen         220
Domino's Pizza      215
SONIC Drive-In      205
Carl's Jr           130
Jimmy Johns         121
Hardees             104
Krystal              82
Dunkin' Donuts       76
Panera Bread         76
SUBWAY               73
Culver's             71
Del Taco             66
Panda Express        62
Steak N Shake        56
In-N-Out Burger      54
Taco John's          53
Name: Name, dtype: int64

In [32]:
# export to csv as this is clean data incase anything happens while added in other informatoin
clean_rest_2018.to_csv('../Resources/clean_rest_2018.csv', encoding='utf-8', index = False)

In [33]:
# combine with chipotle csv
# first read get file then read into pandas DF
Chipotle_stores = "../Resources/chipotle_stores.csv"
chipotle = pd.read_csv(Chipotle_stores)
chipotle.head(2)
print(len(chipotle))

2629


In [34]:
clean_rest_2018.head(2)
print(len(clean_rest_2018))

8526


In [35]:
# merge 2 restaurant dataframes. Used concat to just add the other restaurant data underneath the chipotle information
all_rest_df = pd.concat([chipotle, clean_rest_2018])
print(len(all_rest_df))
all_rest_df

11155


,Name,Address,City,State,Zip Code,Country,Latitude,Longitude
0,Chipotle Mexican Grill,346 W Magnolia Ave,Auburn,Alabama,36832,US,32.606813,-85.487328
1,Chipotle Mexican Grill,300 20th St S,Birmingham,Alabama,35233,US,33.509722,-86.802756
2,Chipotle Mexican Grill,3220 Morrow Rd,Birmingham,Alabama,35235,US,33.595581,-86.647437
3,Chipotle Mexican Grill,4719 Highway 280,Birmingham,Alabama,35242,US,33.422582,-86.698279
4,Chipotle Mexican Grill,1821 Cherokee Ave SW,Cullman,Alabama,35055,US,34.154134,-86.841220
...,...,...,...,...,...,...,...,...
9995,YUM Brands,3460 Robinhood Rd,Winston-Salem,NC,27106,US,36.117563,-80.316553
9996,YUM Brands,3069 Kernersville Rd,Winston-Salem,NC,27107,US,36.077718,-80.176748
9997,YUM Brands,838 S Main St,Kernersville,NC,27284,US,36.111015,-80.089165
9998,YUM Brands,1702 Glendale Dr SW,Wilson,NC,27893,US,35.719981,-77.945795


In [36]:
# Add census data to right of all rest data(printed head to get column names)
census_2018 =clean_2018_census_pd
census_2018.head(2)

,Zip Code,Latitude,Longitude,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate
0,00601,18.180103,-66.74947,17242.0,40.5,13092.0,6999.0,10772.0,62.475351
1,00602,18.363285,-67.18024,38442.0,42.3,16358.0,9277.0,19611.0,51.014515


In [38]:
# Add census data to right of all rest data
all_rest_new = pd.merge(all_rest_df,census_2018, on="Zip Code", how='left')
all_rest_new.head(50)

,Name,Address,City,State,Zip Code,Country,Latitude_x,Longitude_x,Latitude_y,Longitude_y,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate
0,Chipotle Mexican Grill,346 W Magnolia Ave,Auburn,Alabama,36832,US,32.606813,-85.487328,32.589553,-85.539910,24509.0,23.7,27638.0,22390.0,9821.0,40.070994
1,Chipotle Mexican Grill,300 20th St S,Birmingham,Alabama,35233,US,33.509722,-86.802756,33.509623,-86.799700,1715.0,27.7,41500.0,29812.0,655.0,38.192420
2,Chipotle Mexican Grill,3220 Morrow Rd,Birmingham,Alabama,35235,US,33.595581,-86.647437,33.609779,-86.655070,20926.0,38.0,55609.0,25477.0,2829.0,13.519067
3,Chipotle Mexican Grill,4719 Highway 280,Birmingham,Alabama,35242,US,33.422582,-86.698279,33.409582,-86.696840,55122.0,40.9,91966.0,51709.0,3641.0,6.605348
4,Chipotle Mexican Grill,1821 Cherokee Ave SW,Cullman,Alabama,35055,US,34.154134,-86.841220,34.167893,-86.825060,20884.0,38.4,42252.0,23465.0,3202.0,15.332312
5,Chipotle Mexican Grill,1759 Montgomery Hwy,Hoover,Alabama,35244,US,33.378958,-86.803802,33.356981,-86.815770,31930.0,40.5,86971.0,44782.0,1737.0,5.440025
6,Chipotle Mexican Grill,5900 University Dr NW Ste D2,Huntsville,Alabama,35806,US,34.742319,-86.665720,34.748066,-86.683170,21888.0,36.6,68255.0,40270.0,2596.0,11.860380
7,Chipotle Mexican Grill,3871 Airport Blvd,Mobile,Alabama,36608,US,30.675338,-88.143754,30.690440,-88.197010,35591.0,35.3,49068.0,31222.0,5769.0,16.209154
8,Chipotle Mexican Grill,7765 Airport Blvd D100,Mobile,Alabama,36608,US,30.682731,-88.224998,30.690440,-88.197010,35591.0,35.3,49068.0,31222.0,5769.0,16.209154
9,Chipotle Mexican Grill,2560 Berryhill Rd Ste C,Montgomery,Alabama,36117,US,32.359177,-86.162253,32.373714,-86.164430,53841.0,35.2,62753.0,34749.0,6894.0,12.804368


In [39]:
# Remove and rename extra lat/long columns
all_rest_new = all_rest_new.rename(columns={'Latitude_x': 'Latitude', 'Longitude_x':'Longitude'})
all_rest_df = all_rest_new[['Name', 'Address', 'City','State', 'Zip Code','Country', 'Latitude', 'Longitude','Population', 'Median Age', 'Household Income','Per Capita Income', 'Poverty Count', 'Poverty Rate']]
all_rest_df.tail(50)

,Name,Address,City,State,Zip Code,Country,Latitude,Longitude,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate
11105,McDonald's,28670 Northwestern Hwy,Southfield,MI,48034,US,42.499260,-83.303460,13819.0,42.4,45964.0,35148.0,1496.0,10.825675
11106,McDonald's,205 Back River Neck Rd,Essex,MD,21221,US,39.307620,-76.443300,43110.0,37.8,55115.0,26698.0,6135.0,14.231037
11107,McDonald's,205 Back River Neck Rd,Essex,MD,21221,US,39.307620,-76.443300,43110.0,37.8,55115.0,26698.0,6135.0,14.231037
11108,YUM Brands,7306 McClean Blvd,Parkville,MD,21234,US,39.372797,-76.567488,68938.0,40.0,64510.0,35302.0,5612.0,8.140648
11109,YUM Brands,330 Larkin Dr,Monroe,NY,10950,US,41.319444,-74.148181,50895.0,22.4,76637.0,24832.0,12410.0,24.383535
11110,Burger King,2076 State Route 208,Montgomery,NY,12549,US,41.512497,-74.206690,11235.0,41.3,91386.0,35252.0,674.0,5.999110
11111,Dunkin' Donuts,19 Rector St,New York,NY,10006,US,40.707928,-74.013950,3454.0,31.8,176250.0,114611.0,164.0,4.748118
11112,Culver's,W187S7959 Racine Ave,Muskego,WI,53150,US,42.900136,-88.148686,25622.0,44.2,89371.0,40800.0,579.0,2.259777
11113,McDonald's,801 Meadowbrook Rd,Waukesha,WI,53188,US,43.022800,-88.285200,35799.0,39.7,65043.0,36132.0,3404.0,9.508645
11114,McDonald's,3421 Biddle Ave,Wyandotte,MI,48192,US,42.198350,-83.149680,25138.0,42.7,53110.0,30477.0,2922.0,11.623836


In [41]:
# Export to CSV for others to use
all_rest_df.to_csv("../Resources/All_rest_2018.csv", encoding="utf-8", index=False)

## Stock Data Cleaning